## Training a neural network to classify images of MFCCs

### Load dataset

In [1]:

import os

current_dir = os.getcwd()
directory = os.path.dirname(current_dir) + "/datasets/"
csv_files_train = [directory + "/train/" + f for f in os.listdir(directory + "train/") if f.endswith('.csv')]
csv_files_validation = [directory + "/validation/" + f for f in os.listdir(directory + "validation/") if f.endswith('.csv')]

print("training files: ", len(csv_files_train))
print("validation files: ", len(csv_files_validation))


training files:  770
validation files:  210


Test import of csv datasets into tensorflow datasets

import every csv file as a single matrix with one label associated

In [2]:
import tensorflow as tf

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check if GPU is available and being used
print(tf.config.list_physical_devices('GPU'))



2023-08-14 19:28:40.297100: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-14 19:28:41.273176: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TensorFlow version: 2.13.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-08-14 19:28:42.263897: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 19:28:42.291977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 19:28:42.292224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
import numpy as np

# create training and validation datasets
dataset_train = []
dataset_validation = []
labels_train = []
labels_validation = []

# read csv files into lists
# the label (language) is written in the file name

for file in csv_files_train:
    data_array = np.genfromtxt(file, delimiter=',', dtype=np.int8)
    dataset_train.append(data_array)
    
    file_name = os.path.basename(file)
    labels_train.append(file_name[5:8])

for file in csv_files_validation:
    data_array = np.genfromtxt(file, delimiter=',', dtype=np.int8)
    dataset_validation.append(data_array)

    file_name = os.path.basename(file)
    labels_validation.append(file_name[5:8])

print("dataset train size: ", len(dataset_train))
print("dataset validation size: ", len(dataset_validation))
print("labels train size: ", len(labels_train))
print("labels validation size: ", len(labels_validation))

dataset train size:  770
dataset validation size:  210
labels train size:  770
labels validation size:  210


In [4]:
# print size of one element of the dataset: feature size
mfcc_size = dataset_train[0].shape
print ("mfcc_size: ", mfcc_size)

mfcc_size:  (349, 12)


In [5]:
classes = ["ita", "eng"]

# Create a mapping from class names to integer labels
class_to_index = {class_name: index for index, class_name in enumerate(classes)}

# Convert labels to integer labels using the mapping
integer_labels_train = np.array([class_to_index[label] for label in labels_train], dtype=np.int8)
integer_labels_validation = np.array([class_to_index[label] for label in labels_validation], dtype=np.int8)

y_onehot_train = tf.keras.utils.to_categorical(integer_labels_train, num_classes = len(classes)) # one hot encoding
y_onehot_validation = tf.keras.utils.to_categorical(integer_labels_validation, num_classes = len(classes)) # one hot encoding



In [6]:

x_train = tf.reshape(dataset_train, (-1, mfcc_size[0], mfcc_size[1]))
x_validation = tf.reshape(dataset_validation, (-1, mfcc_size[0], mfcc_size[1]))

print("Training features shape:", x_train.shape)
print("Validation features shape:", x_validation.shape)

# create tensorflow dataset from numpy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_onehot_train))
val_dataset = tf.data.Dataset.from_tensor_slices((x_validation, y_onehot_validation))


Training features shape: (770, 349, 12)
Validation features shape: (210, 349, 12)


2023-08-14 19:28:57.947492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 19:28:57.947950: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-14 19:28:57.948250: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
batch_size = 32
num_epochs = 100
learning_rate = 0.001
mfcc_shape = (mfcc_size[0], mfcc_size[1], 1)

# shuffle and batch
train_dataset = train_dataset.shuffle(len(x_train))

# apply batching to the datasets
val_dataset = val_dataset.batch(batch_size)
train_dataset = train_dataset.batch(batch_size)

In [8]:
for image_batch, labels_batch in train_dataset:
	print("MFCC batch input feature shape: ", image_batch.shape)
	print("MFCC labels shape: ", labels_batch.shape)
	break

MFCC batch input feature shape:  (32, 349, 12)
MFCC labels shape:  (32, 2)


In [9]:
from tensorflow.keras import layers, models
from keras.callbacks import EarlyStopping


# Create a basic CNN model
model = models.Sequential([
    #layers.Reshape(( 1247, 12), input_shape=(1247, 12)),
	layers.Conv2D(filters=64, kernel_size=(5, 1), activation='relu', input_shape=mfcc_shape),
	layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=64, kernel_size=(5, 1), activation='relu'),
	layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=32, kernel_size=(3, 1), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 1)),
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.GlobalAveragePooling2D(), 
	layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
	layers.Dense(2, activation='softmax')  # Two classes
])

model.summary()

# Compile the model
model.compile(optimizer='adam',
			  loss='categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
			  metrics=['accuracy'])

# checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy',save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=10)

callbacks_list = [early_stopping]

# Train the model
model.fit(x=train_dataset, epochs=num_epochs, callbacks=callbacks_list, batch_size=batch_size, 
          validation_data=val_dataset)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 345, 12, 64)       384       
                                                                 
 max_pooling2d (MaxPooling2  (None, 172, 12, 64)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 168, 12, 64)       20544     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 84, 12, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 82, 12, 32)        6176      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 41, 12, 32)        0

2023-08-14 19:29:09.491613: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-14 19:29:10.015609: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f0eb4052a70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-14 19:29:10.015638: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1050, Compute Capability 6.1
2023-08-14 19:29:10.020263: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-14 19:29:11.482122: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


25/25 [==============================] - 7s 58ms/step - loss: 0.7709 - accuracy: 0.5104 - val_loss: 0.6781 - val_accuracy: 0.5810
Epoch 2/100
25/25 [==============================] - 1s 24ms/step - loss: 0.6797 - accuracy: 0.5766 - val_loss: 0.6824 - val_accuracy: 0.5000
Epoch 3/100
25/25 [==============================] - 1s 24ms/step - loss: 0.6802 - accuracy: 0.5455 - val_loss: 0.6683 - val_accuracy: 0.5381
Epoch 4/100
25/25 [==============================] - 1s 24ms/step - loss: 0.6757 - accuracy: 0.5844 - val_loss: 0.6523 - val_accuracy: 0.6048
Epoch 5/100
25/25 [==============================] - 1s 24ms/step - loss: 0.6539 - accuracy: 0.6052 - val_loss: 0.6540 - val_accuracy: 0.6048
Epoch 6/100
25/25 [==============================] - 1s 24ms/step - loss: 0.6698 - accuracy: 0.5792 - val_loss: 0.6398 - val_accuracy: 0.6286
Epoch 7/100
25/25 [==============================] - 1s 25ms/step - loss: 0.6364 - accuracy: 0.6364 - val_loss: 0.6133 - val_accuracy: 0.6905
Epoch 8/100
25/25 

In [11]:
# evaluate model on test set

evaluation = model.evaluate(val_dataset, batch_size=32)
evaluation = dict(zip(model.metrics_names, evaluation))
print(evaluation)

7/7 [==============================] - 0s 12ms/step - loss: 2.3383 - accuracy: 0.5810
{'loss': 2.3383290767669678, 'accuracy': 0.5809524059295654}


In [28]:
parent_dir = os.path.dirname(os.getcwd())
filepath = parent_dir + "\\model_lite\\"
model.save(filepath +  "CNN_model")

INFO:tensorflow:Assets written to: C:\Users\HP\Documents\GitHub\Spoken_Language_Recognition_Tensorflow_Embedded\model_lite\CNN_model_h5\assets


INFO:tensorflow:Assets written to: C:\Users\HP\Documents\GitHub\Spoken_Language_Recognition_Tensorflow_Embedded\model_lite\CNN_model_h5\assets


## Model Conversion to Tensorflow Lite

In [50]:
parent_dir = os.path.dirname(os.getcwd())
filepath = parent_dir + "\\model_lite\\"
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(filepath + "CNN_model")
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    #tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]

converter.experimental_enable_resource_variables = True

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# write the converted model into a file
with open(filepath + "CNN_model.tflite", 'wb') as f:
	f.write(tflite_model)

In [51]:
model_path = filepath + "CNN_model.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_conv2d_49_input:0', 'index': 0, 'shape': array([   1, 1247,   12,    1]), 'shape_signature': array([  -1, 1247,   12,    1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall:0', 'index': 28, 'shape': array([1, 2]), 'shape_signature': array([-1,  2]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [52]:
# Assuming single input and output tensors.
input_shape = input_details[0]['shape']
output_shape = output_details[0]['shape']

print(input_shape)
print(output_shape)

[   1 1247   12    1]
[1 2]


In [77]:
random_index = np.random.randint(0, len(val_features))

# Select the random data point using the random index
random_data_point = tf.convert_to_tensor(tf.cast(val_features[random_index], dtype=tf.float32))
random_label = tf.convert_to_tensor(val_labels[random_index])
# Convert the random data point from int8 to float32
input_data_matrix = random_data_point
batch_size = 1

input_data_matrix = tf.reshape(input_data_matrix, (-1, 1247, 12, 1))
print(input_data_matrix)



[[-89  32  76 ...  35  63  49]
 [-84  18  57 ... -11  61  22]
 [-83  14  66 ... -32  56  20]
 ...
 [ -6  87  22 ... -14  14   5]
 [ 11  71  -4 ...  -7   1 -16]
 [ 32  69 -33 ... -17   0   2]]


In [78]:
# Set input data to the interpreter.
interpreter.set_tensor(input_details[0]['index'], input_data_matrix)

# Run inference.
interpreter.invoke()

# Get output data from the interpreter.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

In [79]:
# Process output data.
# For example, if your output is classification probabilities:
predicted_class = np.argmax(output_data)
print("Predicted class:", predicted_class)
print("True label: ", random_label)

Predicted class: 1


In [ ]:

!xxd -i converted_model.tflite > model_data.cc